In [1]:
import numpy as np

In [2]:
data = open('./data/input.txt', 'r').read()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print("data has %d characters, %d unique" % (data_size, vocab_size))
char_to_idx = {ch:i for i, ch in enumerate(chars)}
idx_to_char = {i:ch for i, ch in enumerate(chars)}

data has 5358 characters, 50 unique


In [3]:
#hyperparameters 
hidden_size = 100 #size of hidden layers in neurons
seq_length = 25 #number of steps to unroll the RNN for
learning_rate = 1e-1

In [ ]:
#model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 #hidden to output
bh = np.zeros((hidden_size,1)) #hidden bias
by = np.zeros((vocab_size,1)) #output bias

def lossFun(inputs, targets, hprev):
    '''
    inputs, targets are both integers
    hprev is Hx1 array of initial hidden size
    return the loss, gradients on model parameters, and last hidden state
    '''
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    #forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) #encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) #hidden state
        ys[t] = np.dot(Why, hs[t]) + by #unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) #probabilities of next char
        loss += np.log(ps[t][targets[t],0]) #softmax (cross-entropy loss)
    #backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(xrange(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext
        dhraw = (1 - hs[t]*hs[t]) * dh #backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
        
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out = dparam) # clip to mitigate exploding grads
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
    """
    sample a sequence of integers from the model
    h is memory state, seed_ix is seed letter for the first time step
    """
    x = np.zeros((vocab_size,1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p = p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

n, p = 0, 0

mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

while True:
    #prepare the inputs we aer sweeping from left to right in steps seq_length
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size, 1))
        p =0 # go from start of data
    inputs = [char_to_idx[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_idx[ch] for ch in data[p+1:p+seq_length+1]]
    
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(idx_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt,))
        
    #forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0:
        print('iter %d, loss: %f' % (n, smooth_loss)) #print progress
        
        
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                 [dWxh, dWhh, dWhy, dbh, dby],
                                 [mWxh, mWhh, mWhy, mbh, mby]):
        
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adaguard update
        
    p += seq_length # move data pointer
    n += 1 # interation counter

----
 boqTBuLy
n's wHlA-ctHyi
nsdBq-QmO-aIpdEAAhOa'q?n
d?ejt?"rAszHa mpFkxNlHqev!"-mhHx-k:zjExAcCNuzcbzSC!fysAmrib,I!HNd.dznh
,nCrlg?F?
vNiOE"CQ'x:angQxFHLydnuAtwIxWWb !!TivFbW!Orx:f qkxHwNbekqbu:rSg'aT,!ug 
----
iter 0, loss: 97.604979
----
 ro,HiyotpFi  jeI r mtne doe dot alet "e  e! tFyag
m Hrn soi dlr ag,Btura
n, 
 y  -etpge tre gdoIlse io thrl ock Lob tiy  !tu Wea wetrBeeosn tH fgh. hn  go,  c"  "ltwqo :iy 
be hieeSIe:Lsoi hnpdoe T
,r 
----
iter 100, loss: 78.456155
----
 ?aae
 bayo Cedvn cmeihiro f
  fg e .r ip
ywtt Trt chte     ahde r itl,nr"h tm artoware ra baAlb r  dv  Itn go,  ulwtr aat"at  inwsses-l
t.u o ere mar 'Fsgo e  rp i'yl ' neIQaltticnr eitidr h eo OyO b  
----
iter 200, loss: 63.096677
----
 s nf Neosav uyohacetryztpiistet ionds l r e e oho!h i,v nas qe"' y ladd chen,i
 oo S  e tvdimAorepiodif stWojNhohtlnoerdyd inee-  o h  Cdlbcg cm  t ia  edn ou ri
ljct   ro wdt ,!pgslh m!difoi ed :c rn 
----
iter 300, loss: 49.447390
----
 lte! .inuio-hesgteut ohyriis edul yk ela